# Project aiming to recalculate the MAPS score using gnomAD v3 data

### MAPS (mutability-adjusted proportion of singletons) - the score measures the selection against classes of variants in a population (assumes that the more damaging classes of variants are kept at lower frequencies by natural selection).

In the most basic way, higher values of MAPS indicate increased deleteriousness/increased potential to posess low-frequency variants which, according to the assumption of natrual selection of such low frequency variants, increase the morbitidy by inducing heavily impacting diseases/symptoms/processes onto the host overall well-being.    ("Higher values indicate an enrichment of lower frequency variants, which suggests increased deleteriousness", gnomAD paper).

Such deleterious variants (which as said are low frequency) increase an individual’s susceptibility or predisposition to a certain disease or disorder. When such a variant (or mutation) is inherited, development of symptoms is more likely, but not certain. Also called disease-causing mutation, pathogenic variant, predisposing mutation, and susceptibility gene mutation.

And so in summary, the MAPS score may tell that in a given functionall region this particullar population has an increased deleteriousness (described above). This score is not comparable between cohorts.

### Basic step-by-step guide to calculate MAPS:
    1) Divide variants into functional classes (this is VEP's `consequence`);
    2) Acquire the number of singletons in each of the class and the total number of mutations and calculate the proportions;
    3) To correct for variant's mutational class (transitions, transversions, CpGs), one shall also acquire the singleton proportion but only for synonymous functional class of variants;
    4) Train a linear model on synonymous variation weighted by number of observations in each mutational context (this is the step that allows to correct MAPS score for transitions as they are more common than transversions; mutation rates and estimates downloaded from supplementary_data_10 gnomAD paper);
    5) Use the trained model to regress the expected proportion singleton for each functional variant class;
    6) Acquiring the expected proportion of singletons for each functional class - MAPS? 
   **(6) This is called MAPS in supplementary, but in the code a subtraction is done?? to do**
    
    

## 1. Import packages

In [1]:
import hail as hl
from bokeh.io import output_notebook,show

## 2. Import data

In [2]:
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')

Initializing Hail with default parameters...


2022-09-19 18:56:43 WARN  Utils:69 - Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface en0)
2022-09-19 18:56:43 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-09-19 18:56:44 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20220919-1856-0.2.99-57537fea08d4.log


In [3]:
ht_mu = hl.import_table('data/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)

2022-09-19 18:57:00 Hail: INFO: Reading table to impute column types
2022-09-19 18:57:02 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


## 3. Retrieve the variant annotations

### Also subset the data to draft locally

In [4]:
ht = ht.head(100000)

ht.count()

100000

In [5]:
ht.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
    'faf_meta': array<dict<str, str>> 
    'vep_version': str 
    'vep_csq_header': str 
    'dbsnp_version': str 
    'filtering_model': struct {
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        model_id: str, 
        snv_training_variables: array<str>, 
        indel_training_variables: array<str>
    } 
    'age_distribution': struct {
        bin_edges: array<float64>, 
        bin_freq: array<int32>, 
        n_smaller: int32, 
        n_larger: int32
    } 
    'freq_sample_count': array<int32> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<s

In [6]:
ht.aggregate(hl.agg.counter(ht.vep.most_severe_consequence))

frozendict({'3_prime_UTR_variant': 692, '5_prime_UTR_variant': 5, 'downstream_gene_variant': 4308, 'frameshift_variant': 14, 'inframe_insertion': 1, 'intergenic_variant': 5365, 'intron_variant': 65324, 'mature_miRNA_variant': 51, 'missense_variant': 256, 'non_coding_transcript_exon_variant': 16198, 'regulatory_region_variant': 99, 'splice_acceptor_variant': 56, 'splice_donor_variant': 83, 'splice_region_variant': 548, 'start_lost': 3, 'stop_gained': 7, 'stop_lost': 1, 'synonymous_variant': 103, 'upstream_gene_variant': 6886})

In [84]:
ht.aggregate(hl.agg.counter(ht.vep.context))

frozendict({None: 100000})

## 4. Calculate the raw singleton proportion for each functional variant class

In [31]:
# Calculate n variants per consequence
ht_cons_n = (ht.group_by(ht.vep.most_severe_consequence).aggregate(N_variants = hl.agg.count()))
    

In [32]:
ht_cons_n.show(5)

2022-09-19 19:40:51 Hail: INFO: Ordering unsorted dataset with network shuffle4]


,
most_severe_consequence,N_variants
str,int64
"""3_prime_UTR_variant""",692
"""5_prime_UTR_variant""",5
"""downstream_gene_variant""",4308
"""frameshift_variant""",14
"""inframe_insertion""",1


In [33]:
# Retrieve df of singletons
ht_singletons = ht.filter(ht.info.singleton == 1)

In [34]:
ht_singletons_n = (ht_singletons.group_by(ht_singletons.vep.most_severe_consequence).aggregate(N_singletons = hl.agg.count()))
ht_singletons_n.show(3)

2022-09-19 19:41:47 Hail: INFO: Ordering unsorted dataset with network shuffle4]


,
most_severe_consequence,N_singletons
str,int64
"""3_prime_UTR_variant""",196
"""5_prime_UTR_variant""",1
"""downstream_gene_variant""",1539


In [51]:
# Do the ratio
ht_ps = ht_cons_n.join(ht_singletons_n, how = 'inner')

2022-09-19 20:02:54 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'freq_sample_count' -> 'freq_sample_count_1'
    'freq_meta' -> 'freq_meta_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'faf_meta' -> 'faf_meta_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'vep_version' -> 'vep_version_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'filtering_model' -> 'filtering_model_1'
    'age_distribution' -> 'age_distribution_1'


In [52]:
ht_ps = ht_ps.annotate(ps = ht_ps.N_singletons/ht_ps.N_variants)

In [53]:
# Pick only synonymous variants for correction
ht_ps_synonymous = ht_ps.filter(ht_ps.most_severe_consequence == 'synonymous_variant')

In [55]:
ht_ps_synonymous.show()

2022-09-19 20:05:02 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-19 20:05:35 Hail: INFO: Ordering unsorted dataset with network shuffle4]


,,,
most_severe_consequence,N_variants,N_singletons,ps
str,int64,int64,float64
"""synonymous_variant""",103,29,2.82e-01


## Above till 4. not needed

In [77]:
ht_synonymous = ht.filter(ht.vep.most_severe_consequence == "synonymous_variant")
ht_synonymous = ht_synonymous.annotate(ref=ht_synonymous.alleles[0], alt=ht_synonymous.alleles[1])

In [193]:
ht_mu.show(5)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09
"""AAC""","""A""","""C""",0,1.39e-09
"""AAC""","""A""","""G""",0,5.25e-09


In [99]:
import gnomad.utils.vep

In [130]:
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()

In [131]:
context_table.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
    'vep_help': str 
    'vep_config': str 
----------------------------------------
Row fields:
    'idx': int32 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'old_locus': locus<GRCh38> 
    'old_alleles': array<str> 
    'vep': struct {
        assembly_name: str, 
        allele_string: str, 
        ancestral: str, 
        colocated_variants: array<struct {
            aa_allele: str, 
            aa_maf: float64, 
            afr_allele: str, 
            afr_maf: float64, 
            allele_string: str, 
            amr_allele: str, 
            amr_maf: float64, 
            clin_sig: array<str>, 
            end: int32, 
            eas_allele: str, 
            eas_maf: float64, 
            ea_allele: str, 
            ea_maf: float64, 
            eur_allele: str, 
            eur_maf: float64, 
            exac_adj_allele: str, 
  

In [151]:
context_table_parsed = context_table.select(context_table.context)
context_table_parsed.show(5)

,,
locus,alleles,context
locus<GRCh38>,array<str>,str
chr1:10001,"[""T"",""A""]","""NNNTAAC"""
chr1:10001,"[""T"",""C""]","""NNNTAAC"""
chr1:10001,"[""T"",""G""]","""NNNTAAC"""
chr1:10002,"[""A"",""C""]","""NNTAACC"""
chr1:10002,"[""A"",""G""]","""NNTAACC"""


In [200]:
ht_con = ht.join(context_table_parsed, how = 'inner')

In [206]:
ht_con = ht_con.transmute(context = ht_con.context[2:5])
ht_con = ht_con.annotate(ref=ht_con.alleles[0], alt=ht_con.alleles[1])

In [207]:
ht_con_mu = ht_con.key_by("context", "ref", "alt").join(ht_mu.key_by("context", "ref", "alt"), how = 'inner')

In [ ]:
ht_con_mu.show()

2022-09-20 18:47:52 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-09-20 18:47:53 Hail: INFO: Coerced sorted dataset


In [ ]:
### Compute lm

In [152]:
ht_synonymous_context = ht_synonymous.join(context_table_parsed, how = 'inner')

In [153]:
ht_synonymous_context.show(5)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:69045    | ["A","G"]  |
| chr1:69063    | ["T","C"]  |
| chr1:69072    | ["G","A"]  |
| chr1:69081    | ["G","A"]  |
| chr1:69081    | ["G","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(3,2.34e-04,12832,0),(37,3.83e-04,96706,5),(0,0.00e+00,2786,0),(0,0.00e+... |
| [(771,5.18e-02,14898,2),(6248,6.59e-02,94806,1094),(233,6.10e-02,3818,1),... |
| [(6,3.01e-04,19966,1),(8,7.68e-05,104206,1),(0,0.00e+00,5214,0),(0,0.00e+... |
| [(2,9.15e-05,21864,0),(4,3.78e-05,105880,1),(1,1.68e-04,5960,0),(0,0.00e+... |
| [(186,8.51e-03,21846,5),(1330,1.26e-02,105880,272),(126,2.12e-02,5952,3),... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [26153,3551,5405,1257,10439,912,480,110,23,11,3,0,0,6,1,0,0,2,0,0]           |
| [21081,3908,4992,1687,10896,1142,757,646,684,325,128,150,191,235,105,54,7... |
| [22467,4186,5820,2534,14061,1662,1008,229,78,30,13,2,4,2,0,2,1,0,1,3]        |
| [21514,4246,5840,2332,15173,1826,1101,288,175,75,20,18,42,74,41,12,9,16,2... |
| [21514,4246,5840,2332,15173,1826,1101,288,175,75,20,18,42,74,41,12,9,16,2... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bi

In [170]:
ht_synonymous_context_parsed = ht_synonymous_context.transmute(context = ht_synonymous_context.context[2:5])

In [176]:
ht_synonymous_context_parsed_variants_n = (ht_synonymous_context_parsed.group_by(ht_synonymous_context_parsed.context).aggregate(N_variants = hl.agg.count()))

In [177]:
ht_synonymous_context_parsed_variants_n.show()

2022-09-20 10:47:10 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,
context,N_variants
str,int64
"""AAT""",1
"""ACA""",2
"""ACC""",2
"""ACG""",1
"""AGA""",1
"""AGC""",1
"""AGG""",3
"""ATA""",2


In [178]:
ht_synonymous_context_parsed_singletons = ht_synonymous_context_parsed.filter(ht_synonymous_context_parsed.info.singleton == 1)
ht_synonymous_context_parsed_singletons_n = (ht_synonymous_context_parsed_singletons.group_by(ht_synonymous_context_parsed_singletons.context).aggregate(N_singletons = hl.agg.count()))

ht_synonymous_context_parsed_singletons_n.show()

2022-09-20 10:50:16 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,
context,N_singletons
str,int64
"""AAT""",1
"""ATG""",1
"""CAG""",2
"""CAT""",1
"""CCA""",3
"""CCC""",2
"""CCG""",1
"""CCT""",1


In [179]:
ht_synonymous_context_parsed_ps = ht_synonymous_context_parsed_variants_n.join(ht_synonymous_context_parsed_singletons_n, how = 'inner')
ht_synonymous_context_parsed_ps = ht_synonymous_context_parsed_ps.annotate(ps = ht_synonymous_context_parsed_ps.N_singletons/ht_synonymous_context_parsed_ps.N_variants)

2022-09-20 10:57:12 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'vep_help' -> 'vep_help_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'freq_meta' -> 'freq_meta_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'faf_meta' -> 'faf_meta_1'
    'vep_config' -> 'vep_config_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'grange' -> 'grange_1'
    'vep_version' -> 'vep_version_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'filtering_model' -> 'filtering_model_1'
    'age_distribution' -> 'age_distribution_1'


In [180]:
ht_synonymous_context_parsed_ps.show()

2022-09-20 10:57:37 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-09-20 10:57:51 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,,,
context,N_variants,N_singletons,ps
str,int64,int64,float64
"""AAT""",1,1,1.00e+00
"""ATG""",2,1,5.00e-01
"""CAG""",4,2,5.00e-01
"""CAT""",4,1,2.50e-01
"""CCA""",6,3,5.00e-01
"""CCC""",2,2,1.00e+00
"""CCG""",1,1,1.00e+00
"""CCT""",2,1,5.00e-01


In [186]:
lm_ht = ht_mu.key_by("context").join(ht_synonymous_context_parsed_ps.key_by("context"), how = 'inner')
lm_ht.show(10)

2022-09-20 15:33:23 Hail: INFO: Coerced sorted dataset
2022-09-20 15:34:21 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-09-20 15:35:12 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,,,,,,,
context,ref,alt,methylation_level,mu_snp,N_variants,N_singletons,ps
str,str,str,int32,float64,int64,int64,float64
"""AAT""","""A""","""C""",0,1.27e-09,1,1,1.00e+00
"""AAT""","""A""","""G""",0,7.75e-09,1,1,1.00e+00
"""AAT""","""A""","""T""",0,1.51e-09,1,1,1.00e+00
"""CAG""","""A""","""C""",0,1.62e-09,4,2,5.00e-01
"""CAG""","""A""","""G""",0,5.23e-09,4,2,5.00e-01
"""CAG""","""A""","""T""",0,1.16e-09,4,2,5.00e-01
"""CAT""","""A""","""C""",0,2.00e-09,4,1,2.50e-01
"""CAT""","""A""","""G""",0,1.13e-08,4,1,2.50e-01


In [190]:
lm_ht_res = lm_ht.aggregate(hl.agg.linreg(lm_ht.ps, [1, lm_ht.mu_snp], weight=lm_ht.N_variants).beta)
# R code
# expected_proportion_singleton_lm = lm(singletons/n ~ mu_snp, synonymous_tri, weights = synonymous_tri$n)


2022-09-20 15:37:56 Hail: INFO: Coerced sorted dataset
2022-09-20 15:38:49 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-09-20 15:39:32 Hail: INFO: Ordering unsorted dataset with network shuffle1]


In [191]:
lm_ht_res

[0.4467571825234351, -1023121.0259755343]

In [ ]:
# Get expected number of singletons by applying the model factors
